In [1]:
import pandas as pd
import numpy as np
import os

Información metadata: https://github.com/datos-justicia-argentina/Internos-del-Servicio-Penitenciario-Federal/blob/master/Internos-del-Servicio-Penitenciario-Federal-metadata.md

Datasets: http://datos.jus.gob.ar/dataset/internos-del-servicio-penitenciario-federal-spf

Establecimientos: https://www.argentina.gob.ar/spf/establecimientos

Datos de interés para la transformación:
1) Desde el 03/2018 inclusive, la columna sexo se cambió a género
2) Desde el 07/2018 inclusive, se añaden las columnas provincia_nacimiento_id, unidad_provincia, unidad_provincia_indec_id
3) Desde el 05/2019 inclusive, la columna unidad_provincia_indec_id se cambió a unidad_provincia_id
4) En el dataset de CONDENADOS del 12/2021 la situación procesal figura como PROCESADOS
5) En el dataset de CONDENADOS del 09/2019 se encuentra incluído los datos de PROCESADOS y CONDENADOS de esa fecha
6) Los datasets del 06/2017 tiene invertidas las columnas profesion y juzgado

In [2]:
# lista con el nombre de todos los csv
filenames = os.listdir('datasets/')

# lista vacía que contendrá los dataframes de cada csv
dataframes = []

for i in range(len(filenames)):
    
    if filenames[i] == '.ipynb_checkpoints':
        continue
        
    match filenames[i][13]:
        case 'c':
            anio = filenames[i][24:28]
            mes = filenames[i][28:30]
        case 'i':
            anio = filenames[i][26:30]
            mes = filenames[i][30:32]
        case 'p':
            anio = filenames[i][24:28]
            mes = filenames[i][28:30]
        
    df = pd.read_csv('datasets/' + filenames[i])
    
    # Columnas que se modifican para mantener homogeneidad
    
    df.columns = df.columns.str.lower()

    if 'sexo' in df.columns:
        df = df.rename(columns={'sexo': 'genero'})

    if 'ipu' in df.columns:
        df = df.rename(columns={'ipu': 'lpu'})

    if 'unidad_provincia_indec_id' in df.columns:
        df = df.rename(columns={'unidad_provincia_indec_id':'unidad_provincia_id'})
        
    if 'provincia_nacimiento_indec_id' in df.columns:
        df = df.rename(columns={'provincia_nacimiento_indec_id':'provincia_nacimiento_id'})

    if 'jurisdiccion ' in df.columns:
        df = df.rename(columns={'jurisdiccion ':'jurisdiccion'})
        
    # Se corrigen errores específicos por mala introducción de datos
    
    if anio=='2017' and mes=='06':
        df = df.rename(columns={'juzgado':'temp1'})
        df = df.rename(columns={'profesion':'juzgado'})
        df = df.rename(columns={'temp1':'profesion'})
        
    if filenames[i][13] == 'c' and anio == '2021' and mes == '12':
        df['situacion_procesal'] = 'CONDENADO'
    
    if filenames[i][13] == 'c' and anio == '2019' and mes == '09':
        df = df[df["situacion_procesal"] == "CONDENADO"]
        
    if 'unnamed: 22' in df.columns:
        df = df[df["unnamed: 22"].isnull()]
        df = df.drop(columns=["unnamed: 22"])
    
    if 'unnamed: 19' in df.columns:
        df = df[df["unnamed: 19"].isnull()]
        df = df.drop(columns=["unnamed: 19"])
        
    # Se modifican las filas para dar homogeneidad
    
    df = df.apply(lambda x: x.astype(str).str.upper())
    df = df.apply(lambda x: x.astype(str).str.strip())
    
    # Se crea la columna fecha que incluye el mes y año del dataset
        
    df['fecha'] = mes+'/'+anio
    
    dataframes.append(df.copy())
    

df_internos = pd.concat(dataframes, axis=0, ignore_index=True)
# df_internos = df_internos.dropna(axis=1, how='all')

# Columnas que se eliminan porque se encuentran fuera del dominio del problema
df_internos = df_internos.drop(columns=[
                                        # "edad",
                                        "lpu",
                                        "nombre",
                                        "apellido",
                                        "subgrupo",
                                        "juzgado",
                                        "unidad_provincia",
                                        "unidad_provincia_id",
                                        "anios_pena",
                                        "meses_pena",
                                        "dias_pena",
                                        "provincia_nacimiento",
                                        "provincia_nacimiento_id",
                                        "fecha_nacimiento",
                                        "fecha_sentencia_firme"
                                       ])

In [3]:
display(pd.DataFrame(df_internos.columns.values.tolist(), columns=["Columnas"]))

,Columnas
0,unidad
1,situacion_procesal
2,delito
3,edad
4,nacionalidad
5,genero
6,jurisdiccion
7,tipo_pena
8,estado_civil
9,profesion


In [4]:
for i in range(len(df_internos)):
    palabra = str(df_internos.at[i, 'edad'])
    if len(palabra) > 2:
        df_internos.at[i, "edad"] = palabra.rstrip(palabra[-1])

# Columna genero

df_internos = df_internos.loc[df_internos["genero"].isin(["MASCULINO", "FEMENINO", "TRANS"])]

# Columna jurisdiccion

df_internos.loc[df_internos["jurisdiccion"] == "NAN", "jurisdiccion"] = "NO ESPECIFICA"
df_internos.loc[df_internos["jurisdiccion"] == 'NO CONSTA', "jurisdiccion"] = "OTRA JURISDICCION"
df_internos = df_internos.loc[df_internos["jurisdiccion"].isin(["NACIONAL", "FEDERAL", "PROVINCIAL", "OTRA JURISDICCION", "NO ESPECIFICA"])]

# Columna tipo_pena

df_internos.loc[df_internos["tipo_pena"].isin(["NAN","0"]), "tipo_pena"] = "NO ESPECIFICA"
df_internos.loc[df_internos["tipo_pena"] == "PRISIO N", "tipo_pena"] = "PRISION"
df_internos.loc[df_internos["tipo_pena"].isin(["PRISION DISC.", "PRISION DISC"]), "tipo_pena"] = "PRISION DISC"
df_internos = df_internos.loc[df_internos["tipo_pena"].isin(["PRISION PERPETUA", "NO ESPECIFICA", "RECLUSION PERPETUA", "PRISION",
                                                            "RECLUSION", "PRISION DISCONTINUA"])]

# Columna fecha_ingreso

df_internos.loc[df_internos["fecha_ingreso"] == "NAN", "fecha_ingreso"] = ""

# Columna estado_civil

df_internos.loc[df_internos["estado_civil"] == "NAN", "estado_civil"] = "NO ESPECIFICA"

# Columna delito

df_internos.loc[df_internos["delito"].isin(["LEY Nº23737 - TENENCIA Y TRÁFICO DE ESTUPEFACIENTES","LEY 23.737","LEY 23737","LEY Nº23.737",
                                           "LEY Nº23737","LEY N°23737","LEY NÂ°23737 - TENENCIA Y TRÃ¡FICO DE ESTUPEFACIENTES",
                                           'LEY N°23737 - TENENCIA Y TRÁFICO DE ESTUPEFACIENTES - LEY N°23737 - TENENCIA Y TRÁFICO DE ESTUPEFACIENTES',
                                           'LEY N°23737 - TENENCIA Y TRÁFICO DE ESTUPEFACIENTES', "LEY N°23.737"]), "delito"] = "TENENCIA Y TRÁFICO DE ESTUPEFACIENTES"
df_internos.loc[df_internos["delito"].isin(["PROPIEDAD"]), "delito"] = "CONTRA LA PROPIEDAD"
df_internos.loc[df_internos["delito"].isin(["PERSONAS"]), "delito"] = "CONTRA LAS PERSONAS"
df_internos.loc[df_internos["delito"].isin(["NO CONSTA1","NAN","NO COSTA"]), "delito"] = "NO CONSTA"
df_internos.loc[df_internos["delito"].isin(["LIBERTAD"]), "delito"] = "CONTRA LA LIBERTAD"
df_internos.loc[df_internos["delito"].isin(["HONESTIDAD"]), "delito"] = "CONTRA LA HONESTIDAD"
df_internos.loc[df_internos["delito"].isin(["ADMINISTRACION PUBLICA","CONTRA LA ADMINISTRACION PUBLICA", 'CONTRA LA ADMINISTRACIÃ³N PÃºBLICA']), "delito"] = "CONTRA LA ADMINISTRACIÓN PÚBLICA"
df_internos.loc[df_internos["delito"].isin(["SEGURIDAD PUBLICA","CONTRA LA SEGURIDAD PUBLICA",'CONTRA LA SEGURIDAD PÃºBLICA']), "delito"] = "CONTRA LA SEGURIDAD PÚBLICA"
df_internos.loc[df_internos["delito"].isin(["LEY Nº22415 - CODIGO ADUANERO", 'LEY N°22415 - CODIGO ADUANERO','LEY 22415']), "delito"] = "CÓDIGO ADUANERO"
df_internos.loc[df_internos["delito"].isin(["SEGURIDAD NACIONAL","CONTRA LA SEGURIDAD DE LA NACIÓN",'CONTRA LA SEGURIDAD DE LA NACIÃ³N',
                                            'LEY N°13985 - DELITOS CONTRA LA SEGURIDAD DE LA NACION']), "delito"] = "CONTRA LA SEGURIDAD NACIONAL"
df_internos.loc[df_internos["delito"].isin(["FE PUBLICA","CONTRA LA FE PUBLICA","CONTRA LA FÉ PÚBLICA",'CONTRA LA FE PÃºBLICA','CONTRA LA FÃ© PÃºBLICA']), "delito"] = "CONTRA LA FE PÚBLICA"
df_internos.loc[df_internos["delito"].isin(["CONTRA EL ORDEN ECONOMICO Y FINANCIERO","CONTRA EL ORDEN ECONOMICO","ORDEN ECONOMICO Y FINANCIERO",
                                            'CONTRA EL ORDEN ECONÃ³MICO Y FINANCIERO']), "delito"] = "CONTRA EL ORDEN ECONÓMICO Y FINANCIERO"
df_internos.loc[df_internos["delito"].isin(["ESTADO CIVIL"]), "delito"] = "CONTRA EL ESTADO CIVIL"
df_internos.loc[df_internos["delito"].isin(["CONTRA EL ORDEN PUBLICO","ORDEN PUBLICO",'CONTRA EL ORDEN PÃºBLICO']), "delito"] = "CONTRA EL ORDEN PÚBLICO"
df_internos.loc[df_internos["delito"].isin(["LEY N°6582/58 DOMINIO E IDENTIFICACION DE AUTOMOTORES"]), "delito"] = "DOMINIO E IDENTIFICACIÓN DE AUTOMOTORES"
df_internos.loc[df_internos["delito"].isin(["LEY N°26842 - TRATA DE PERSONAS"]), "delito"] = "TRATA DE PERSONAS"

# Columna unidad

df_internos.loc[df_internos["unidad"].isin(["CABA","C.A.B.A.","C.P.F. C.A.B.A.", "CPF"]), "unidad"] = "CPF CABA"
df_internos.loc[df_internos["unidad"].isin(["CPFI","CPF I","C.P.F. I",'CPF I - SPPV']), "unidad"] = "CPF 1"
df_internos.loc[df_internos["unidad"].isin(["CPF II","CPFII","C.P.F. II","PFII"]), "unidad"] = "CPF 2"
df_internos.loc[df_internos["unidad"].isin(["CPF III","CPFIII","CPFIII NOA VA","C.P.F. III","C.P.F. III - V","C.P.F. III - VARONES",
                                            "C.P.F. III - MUJERES",'CPF III - VARONES','CPF III - MUJERES','CPF III - V','CPF III - F',
                                           "C.P.F. III - F","C.P.F. III - M",'CPF III - M','CPFIII NOA M',"CPFIII NOA"]), "unidad"] = "CPF 3"
df_internos.loc[df_internos["unidad"].isin(["CPF IV","CPFIV","C.P.F. IV"]), "unidad"] = "CPF 4"
df_internos.loc[df_internos["unidad"].isin(["CPF V","CPFV",'ANEXO SENILLOSA - U.9','U.9- AS','U9 - AS','U9 AS','U. 9 - AS','ANEXO SENILLOSA',
                                            'U. 9 - AS','U9 SENILLOSA','U-9 ANEXO','U-9 ANEXO','SENILLOSA','U 9']), "unidad"] = "CPF 5"
df_internos.loc[df_internos["unidad"].isin(["CPF VI","CPFVI"]), "unidad"] = "CPF 6"
for i in range(40):
    df_internos.loc[df_internos["unidad"].isin(["U." + str(i),"U-" + str(i),"U" + str(i),"U. " + str(i)]), "unidad"] = "U " + str(i)
df_internos.loc[df_internos["unidad"].isin(['U 19 - U 33','U-19 (U-33)','U-19- U-33','U-19-U-33','U-19-U33','U.19 - U.33','U19 - U33',
                                           'U.19-U33','U19- U33','U-19 - U-33','U-19 U-33']), "unidad"] = "U 19"
df_internos.loc[df_internos["unidad"].isin(['C.F.J.A',"CFJA - ANEXO",'CFJA - II','CFJA- ANEXO','C.F.J.A.', 'U-24 ANEXO', 'U 24']), "unidad"] = "CFJA"
df_internos.loc[df_internos["unidad"].isin(['U-13 CASA PR.']), "unidad"] = "U 13"

# Columna edad

df_internos.loc[df_internos["edad"].isin(['NAN']), "edad"] = ""
df_internos = df_internos.loc[df_internos["edad"] >= '16']

#Columna nacionalidad

df_internos.loc[df_internos["nacionalidad"].isin(['NAN']), "nacionalidad"] = "NO ESPECIFICA"
df_internos.loc[df_internos["nacionalidad"].isin(['PERU - ARGENTINA', "PERU", 'ARGENTINA - PERU']), "nacionalidad"] = "PERÚ"
df_internos.loc[df_internos["nacionalidad"].isin(['REPÃºBLICA DOMINICANA', 'REPÃºBLICA DOMINICANA - ARGENTINA', 'REPÚBLICA DOMINICANA - ARGENTINA']), "nacionalidad"] = "REPÚBLICA DOMINICANA"
df_internos.loc[df_internos["nacionalidad"].isin(['ESPAÃ±A']), "nacionalidad"] = "ESPAÑA"
df_internos.loc[df_internos["nacionalidad"].isin(['REPÃºBLICA DE KOREA', "REPÚBLICA DE KOREA", 'REPÚBLICA DEL PUEBLO DEMOCRÁTICO DE KOREA']), "nacionalidad"] = "REPÚBLICA DE COREA"
df_internos.loc[df_internos["nacionalidad"].isin(['TURQUÃ\xadA']), "nacionalidad"] = "TURQUÍA"
df_internos.loc[df_internos["nacionalidad"].isin(['FEDERACIÃ³N RUSA']), "nacionalidad"] = "FEDERACIÓN RUSA"
df_internos.loc[df_internos["nacionalidad"].isin(['BÃ©LGICA']), "nacionalidad"] = "BÉLGICA"
df_internos.loc[df_internos["nacionalidad"].isin(['ARGENTINA - PARAGUAY', 'PARAGUAY - ARGENTINA']), "nacionalidad"] = "PARAGUAY"
df_internos.loc[df_internos["nacionalidad"].isin(['ARGENTINA - BOLIVIA']), "nacionalidad"] = "BOLIVIA"
df_internos.loc[df_internos["nacionalidad"].isin(['REPÃºBLICA CENTROAFRICANA']), "nacionalidad"] = "REPÚBLICA CENTROAFRICANA"
df_internos.loc[df_internos["nacionalidad"].isin(['ESTADOS UNIDOS DE LA AMÃ©RICA DEL NORTE', "ESTADOS UNIDOS DE LA AMÉRICA DEL NORTE"]), "nacionalidad"] = "ESTADOS UNIDOS"
df_internos.loc[df_internos["nacionalidad"].isin(['REPÃºBLICA ISLÃ¡MICA DE IRAN']), "nacionalidad"] = "REPÚBLICA ISLÁMICA DE IRAN"
df_internos.loc[df_internos["nacionalidad"].isin(['HUNGRÃ\xadA']), "nacionalidad"] = "HUNGRÍA"
df_internos.loc[df_internos["nacionalidad"].isin(['COLOMBIA - ARGENTINA']), "nacionalidad"] = "COLOMBIA"
df_internos.loc[df_internos["nacionalidad"].isin(['ARGENTINA - CHILE']), "nacionalidad"] = "CHILE"
df_internos.loc[df_internos["nacionalidad"].isin(['ARGENTINA - DOMINICA']), "nacionalidad"] = "DOMINICA"
df_internos.loc[df_internos["nacionalidad"].isin(['REPÚBLICA DEMOCRÁTICA DEL CONGO']), "nacionalidad"] = "CONGO"

# Columna profesion

df_internos["profesion"] = df_internos["profesion"].str.replace("\'","")
df_internos.loc[df_internos["profesion"].isin(["", "NAN"]), "profesion"] = "NO ESPECIFICA"
df_internos.loc[df_internos["profesion"].isin(["CARECE", "SIN GRUPO","NO CONSTA"]), "profesion"] = "DESOCUPADO"
df_internos.loc[df_internos["profesion"].isin(['ALBAÃ‘IL']), "profesion"] = "ALBAÑIL"
df_internos.loc[df_internos["profesion"].isin(['AYUDANTE DE ALBAÃ‘IL',]), "profesion"] = "AYUDANTE DE ALBAÑIL"
df_internos.loc[df_internos["profesion"].isin(['NIÃ‘ERA',]), "profesion"] = "NIÑERA"
df_internos.loc[df_internos["profesion"].isin(['DISEÃ‘ADOR GRAFICO',]), "profesion"] = "DISEÑADOR GRÁFICO"
df_internos.loc[df_internos["profesion"].isin(['LEÃ‘ERO',]), "profesion"] = "LEÑERO"

In [5]:
df_internos.to_csv('internos.csv', index=False);